In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
FOLDERNAME = 'Colab\ Notebooks/3_Split'

In [3]:
%cd drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/Colab Notebooks/3_Split


In [4]:
%ls

backup_000200.pth  backup_001600.pth  backup_003000.pth  backup_004400.pth
backup_000400.pth  backup_001800.pth  backup_003200.pth  backup_004600.pth
backup_000600.pth  backup_002000.pth  backup_003400.pth  backup_004800.pth
backup_000800.pth  backup_002200.pth  backup_003600.pth  Test/
backup_001000.pth  backup_002400.pth  backup_003800.pth  Train/
backup_001200.pth  backup_002600.pth  backup_004000.pth  Val/
backup_001400.pth  backup_002800.pth  backup_004200.pth


In [5]:
import os
import glob
import shutil
import pandas as pd

In [ ]:
multiple = pd.read_csv('multiple_times_number.csv')
multiple['編號'] = multiple['編號'].astype(str)
len(multiple)

219

In [7]:
files = glob.glob("*/*.jpg")
len(files)

2435

In [ ]:
df = pd.DataFrame(
    dict(
        category=[f.split("/")[1] for f in files],
        image_path=files,
        id = [f.split("/")[2].split('_')[1].split('.')[0] for f in files]
    )
)
df

,category,image_path,id
0,缺血,Processed/缺血/b_1987.jpg,1987
1,缺血,Processed/缺血/b_999.jpg,999
2,缺血,Processed/缺血/b_1030.jpg,1030
3,缺血,Processed/缺血/b_2120.jpg,2120
4,缺血,Processed/缺血/b_216.jpg,216
...,...,...,...
2430,出血,Processed/出血/b_673.jpg,673
2431,出血,Processed/出血/b_1808.jpg,1808
2432,出血,Processed/出血/b_407.jpg,407
2433,出血,Processed/出血/b_645.jpg,645


In [ ]:
df['subset'] = 'train'

In [ ]:
df.loc[df['id'].isin(multiple['編號']),'subset'] = '_train'
print(len(df.loc[df['subset'] =='_train']))
print(len(df.loc[df['subset'] =='train']))

187
2248


In [ ]:
category = ['出血', '缺血', '無中風']
for s in category:
  sub_n = len(df.loc[df.category==s])
  df.loc[df.query(f"category=='{s}' and subset == 'train'").sample(n=int(sub_n*0.1)).index, 'subset'] = 'val'
for s in category:
  sub_n = len(df.loc[df.category==s])
  df.loc[df.query(f"category=='{s}' and subset == 'train'").sample(n=int(sub_n*0.1)).index, 'subset'] = 'test'

In [ ]:
print(df.loc[df.subset == 'train'].shape)
print(df.loc[df.subset == '_train'].shape)
print(df.loc[df.subset == 'val'].shape)
print(df.loc[df.subset == 'test'].shape)

(1762, 4)
(187, 4)
(243, 4)
(243, 4)


In [ ]:
df.loc[df.subset=='_train', 'subset'] = 'train'

In [ ]:
print(df.loc[df.subset == 'train'].shape)
print(df.loc[df.subset == '_train'].shape)
print(df.loc[df.subset == 'val'].shape)
print(df.loc[df.subset == 'test'].shape)

(1949, 4)
(0, 4)
(243, 4)
(243, 4)


In [ ]:
df['_category'] = 'N'
df.loc[df.category == '出血', '_category'] = 'ICH'
df.loc[df.category == '缺血', '_category'] = 'Ischemic'
df.loc[df.category == '無中風', '_category'] = 'Non-stroke'
df

,category,image_path,id,subset,_category
0,缺血,Processed/缺血/b_1987.jpg,1987,train,Ischemic
1,缺血,Processed/缺血/b_999.jpg,999,train,Ischemic
2,缺血,Processed/缺血/b_1030.jpg,1030,train,Ischemic
3,缺血,Processed/缺血/b_2120.jpg,2120,train,Ischemic
4,缺血,Processed/缺血/b_216.jpg,216,train,Ischemic
...,...,...,...,...,...
2430,出血,Processed/出血/b_673.jpg,673,train,ICH
2431,出血,Processed/出血/b_1808.jpg,1808,test,ICH
2432,出血,Processed/出血/b_407.jpg,407,train,ICH
2433,出血,Processed/出血/b_645.jpg,645,train,ICH


In [ ]:
os.makedirs("train", exist_ok=True)
os.makedirs("val", exist_ok=True)
os.makedirs("test", exist_ok=True)

for i in range(len(df)):
  img_path, category, subset = df.iloc[i][["image_path", "category", "subset"]]
  img_new_path = os.path.join(subset, os.path.basename(img_path))
  shutil.copy(img_path, img_new_path)
  label_new_path = img_new_path.replace(".jpg", ".txt")
  with open(label_new_path, 'w') as f:
    f.write(category)


In [ ]:
d_files = glob.glob("3_Split/Train/*.jpg")
len(d_files)

1949

In [ ]:
for f in d_files:
  os.remove(f)

In [ ]:
for i in range(len(df)):
  img_path, category, subset = df.iloc[i][["image_path", "_category", "subset"]]
  img_new_path = os.path.join(subset, os.path.basename(img_path))
  label_new_path = img_new_path.replace(".jpg", ".txt")
  with open(label_new_path, 'w') as f:
    f.write(category)

In [ ]:
df.groupby(by=['_category', 'subset']).count()

category  image_path    id
_category  subset                            
ICH        test          30          30    30
           train        240         240   240
           val           30          30    30
Ischemic   test         139         139   139
           train       1117        1117  1117
           val          139         139   139
Non-stroke test          74          74    74
           train        592         592   592
           val           74          74    74

### oversample